In [2]:
import leafmap
import streamlit
import geopandas as gpd

In [33]:
farms_df = gpd.read_file(r"data\vector\ce_farms.gpkg")
farms = farms_df[['farmer', 'crop', 'district', 'province', 'area_ha', 'geometry']]
farms.head()

,farmer,crop,district,province,area_ha,geometry
0,Killarney Farm,Groundnut,Chibombo,Central,20.852368,"MULTIPOLYGON (((28.40682 -14.81873, 28.40445 -..."
1,ADM Farm,Groundnut,Chibombo,Central,53.496625,"MULTIPOLYGON (((28.29156 -14.81992, 28.29139 -..."
2,Simon Hunt,Groundnut,Mpongwe,Copperbelt,19.604325,"MULTIPOLYGON (((28.31578 -13.61503, 28.322 -13..."
3,Isranet Farm,Groundnut,Mpongwe,Copperbelt,49.826289,"MULTIPOLYGON (((28.41546 -13.6795, 28.41705 -1..."
4,Matulufu Farm,Groundnut,Luanshya,Copperbelt,30.613994,"MULTIPOLYGON (((28.49302 -13.18086, 28.49362 -..."


In [41]:
selected_farm_name = "ADM Farm"

selected_farm = farms[farms["farmer"] == selected_farm_name]
selected_farm

,farmer,crop,district,province,area_ha,geometry
1,ADM Farm,Groundnut,Chibombo,Central,53.496625,"MULTIPOLYGON (((28.29156 -14.81992, 28.29139 -..."
15,ADM Farm,Groundnut,Chibombo,Central,25.921008,"MULTIPOLYGON (((28.29315 -14.81159, 28.2955 -1..."


In [44]:
m = leafmap.Map(center= [-13.2, 28], zoom= 6.5, height= "1000px")
m.add_basemap("ROADMAP")
m.add_gdf(selected_farm, zoom_to_layer=True, layer_name="Farms")
# m.to_streamlit(height=700)
m

Map(center=[-13.2, 28], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…